In [6]:
import sqlite3
import pandas as pd

# Define the database name
DB_NAME = '../data/Neutrino.db'

# Connect to the database
conn = sqlite3.connect(DB_NAME)

# Query to retrieve all data from the SensorsData table
query = "SELECT * FROM SensorsData ORDER BY TimeStamp DESC LIMIT 10"

# Use pandas to read the data into a DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the DataFrame
df

,id,TimeStamp,Temperature,Gas,Humidity,Pressure,Altitude,Luminosity,soil_moisture,soil_temperature
0,2,2025-02-07 00:33:58.311051,68.80,129,72.44,1008.07,1046.21,99,676,59.79
1,1,2025-02-07 00:16:12.446482,75.00,100,50.00,1000.00,1000.00,105,400,25.00
2,3,2025-02-06 23:33:58.311051,67.14,130,56.33,991.57,1020.55,81,494,56.26
3,4,2025-02-06 22:33:58.311051,74.11,111,53.82,1009.79,1004.06,138,676,62.63
4,5,2025-02-06 21:33:58.311051,77.84,108,47.33,1006.40,1026.14,81,446,75.12
5,6,2025-02-06 20:33:58.311051,75.43,123,41.74,997.55,991.36,69,539,60.69
6,7,2025-02-06 19:33:58.311051,83.95,148,30.01,1014.77,950.63,156,608,55.15
7,8,2025-02-06 18:33:58.311051,83.63,139,62.14,1019.44,970.72,130,340,56.84
8,9,2025-02-06 17:33:58.311051,65.34,93,46.15,991.53,1040.28,106,462,51.70
9,10,2025-02-06 16:33:58.311051,67.99,108,59.02,1016.40,1047.92,133,480,77.10


In [17]:
import sqlite3

DB_NAME = '../data/Neutrino.db'

def fetch_latest_data():
    """Fetch the most recent sensor data."""
    try:
        conn = sqlite3.connect(DB_NAME)
        curs = conn.cursor()
        curs.execute("SELECT * FROM SensorsData ORDER BY timestamp DESC LIMIT 1")
        data = curs.fetchone()
        conn.close()
        print("Fetched data:", data)  # Debugging
        return data
    except sqlite3.Error as e:
        print(f"Error fetching data: {e}")
        return None

fetch_latest_data()

Fetched data: (1, '2025-02-07 00:16:12.446482', 75, 100, 50, 1000, 1000, 105, 400, 25)


(1, '2025-02-07 00:16:12.446482', 75, 100, 50, 1000, 1000, 105, 400, 25)

In [4]:
import os
import logging
from datetime import datetime, timedelta
import pytz
import random
from sqlalchemy import create_engine, Column, Integer, Numeric, DateTime
from sqlalchemy.orm import declarative_base, sessionmaker

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define SQLAlchemy Base
Base = declarative_base()

# Database Setup
data_folder = "../data"
db_name = "Neutrino.db"
db_path = f"sqlite:///{data_folder}/{db_name}"

# Ensure data folder exists
os.makedirs(data_folder, exist_ok=True)

# Define the SensorsData Table
class Sensors(Base):
    __tablename__ = "SensorsData"
    id = Column(Integer, primary_key=True)  # ✅ Auto-increment primary key (no need to specify manually)
    TimeStamp = Column(DateTime)
    Temperature = Column(Numeric)
    Gas = Column(Numeric)
    Humidity = Column(Numeric)
    Pressure = Column(Numeric)
    Altitude = Column(Numeric)
    Luminosity = Column(Numeric)
    soil_moisture = Column(Numeric)
    soil_temperature = Column(Numeric)

# Create Database Engine
engine = create_engine(db_path, echo=False)
Base.metadata.create_all(engine)

# Create a session
Session = sessionmaker(bind=engine)
session = Session()

# Function to generate random sensor values
def generate_sensor_data():
    return {
        "Temperature": round(random.uniform(60, 85), 2),  # °F
        "Gas": random.randint(90, 150),  # Arbitrary gas level
        "Humidity": round(random.uniform(30, 80), 2),  # Percentage
        "Pressure": round(random.uniform(980, 1025), 2),  # hPa
        "Altitude": round(random.uniform(950, 1050), 2),  # Meters
        "Luminosity": random.randint(50, 200),  # Lux
        "soil_moisture": random.randint(300, 700),  # Arbitrary units
        "soil_temperature": round(random.uniform(50, 80), 2)  # °F
    }

# Function to insert fake data for the past 7 days (hourly)
def insert_fake_data(session):
    est = pytz.timezone("US/Eastern")
    now = datetime.now(est)

    data_entries = []

    for days_ago in range(7):
        for hour in range(24):
            timestamp = now - timedelta(days=days_ago, hours=hour)
            sensor_values = generate_sensor_data()

            data_entries.append(
                Sensors(
                    TimeStamp=timestamp,
                    Temperature=sensor_values["Temperature"],
                    Gas=sensor_values["Gas"],
                    Humidity=sensor_values["Humidity"],
                    Pressure=sensor_values["Pressure"],
                    Altitude=sensor_values["Altitude"],
                    Luminosity=sensor_values["Luminosity"],
                    soil_moisture=sensor_values["soil_moisture"],
                    soil_temperature=sensor_values["soil_temperature"],
                )
            )

    session.bulk_save_objects(data_entries)  # ✅ Use bulk insert for efficiency
    session.commit()
    logging.info("✅ Fake sensor data inserted successfully!")

# Run Data Insertion
insert_fake_data(session)

# Verify Data Inserted
import pandas as pd
df = pd.read_sql("SELECT * FROM SensorsData ORDER BY TimeStamp DESC LIMIT 10", engine)
df


2025-02-07 00:33:58,314 - INFO - ✅ Fake sensor data inserted successfully!


,id,TimeStamp,Temperature,Gas,Humidity,Pressure,Altitude,Luminosity,soil_moisture,soil_temperature
0,2,2025-02-07 00:33:58.311051,68.80,129,72.44,1008.07,1046.21,99,676,59.79
1,1,2025-02-07 00:16:12.446482,75.00,100,50.00,1000.00,1000.00,105,400,25.00
2,3,2025-02-06 23:33:58.311051,67.14,130,56.33,991.57,1020.55,81,494,56.26
3,4,2025-02-06 22:33:58.311051,74.11,111,53.82,1009.79,1004.06,138,676,62.63
4,5,2025-02-06 21:33:58.311051,77.84,108,47.33,1006.40,1026.14,81,446,75.12
5,6,2025-02-06 20:33:58.311051,75.43,123,41.74,997.55,991.36,69,539,60.69
6,7,2025-02-06 19:33:58.311051,83.95,148,30.01,1014.77,950.63,156,608,55.15
7,8,2025-02-06 18:33:58.311051,83.63,139,62.14,1019.44,970.72,130,340,56.84
8,9,2025-02-06 17:33:58.311051,65.34,93,46.15,991.53,1040.28,106,462,51.70
9,10,2025-02-06 16:33:58.311051,67.99,108,59.02,1016.40,1047.92,133,480,77.10
